In [3]:
import ast
import bs4
import time
import json
import requests
import pandas as pd
import scipy as scipy
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime
import matplotlib.pyplot as plt

%matplotlib inline

In [121]:
df = pd.read_csv('canna_df.csv')
df['Feelings'] = df['Feelings'].map(ast.literal_eval) ## To get dicts instead of strings
df['Cannabinoids'] = df['Cannabinoids'].map(ast.literal_eval)

In [122]:
## Open dictionaries in columns ['Feelings','Cannabinoids'] to columns

df = df.reset_index(drop=True)


for i in range(5): 
    df[f'Feeling_{i+1}'] = df.apply(lambda x: x['Feelings']['Feelings'][i],axis=1) ## Turn feelings list to columns
    df[f'Negative_{i+1}'] = df.apply(lambda x: x['Feelings']['Negatives'][i],axis=1) ## Turn negative feelings list to columns
    df[f'Helps with_{i+1}'] = df.apply(lambda x: x['Feelings']['Helps with'][i],axis=1) ## Turn helps with list to columns

    df[f'Feeling_{i+1}'] = df[f'Feeling_{i+1}'].str.replace(r'\b $','',regex=True) ## Remove space at the end of values
    df[f'Negative_{i+1}'] = df[f'Negative_{i+1}'].str.replace(r'\b $','',regex=True)
    df[f'Helps with_{i+1}'] = df[f'Helps with_{i+1}'].str.replace(r'\b $','',regex=True)


df['THC'] = df.apply(lambda x: x['Cannabinoids']['THC'],axis=1) ## Turn Cannabinoid dict to columns
df['CBD'] = df.apply(lambda x: x['Cannabinoids']['CBD'],axis=1) 
df['CBG'] = df.apply(lambda x: x['Cannabinoids']['CBG'],axis=1) 


df.drop(['Feelings','Cannabinoids'],inplace=True,axis=1) ## Remove old columns

cols = ['Rating','Rating Users','THC','CBD','CBG']

df['THC'] = df['THC'].str.replace("%",'',regex=True) # Remove % from numbers
df['CBD'] = df['CBD'].str.replace("%",'',regex=True)
df['CBG'] = df['CBG'].str.replace("%",'',regex=True)


df['CBD'].replace("—",np.nan,regex=True,inplace=True) # Replace '-' values with NaN
df['THC'].replace("—",np.nan,regex=True,inplace=True)
df['CBG'].replace("—",np.nan,regex=True,inplace=True)

df[cols] = df[cols].apply(pd.to_numeric) # Change columns type to numeric





In [ ]:
df['Type'].value_counts()

In [64]:
## Fill missing THC values

def my_func(df,parent):

    func_df = df.loc[(df['Left Parent'] == parent) | (df['Right parent'] == parent) | (df['Strain Name'] == parent)] ## All rows with same parent, including the parent
    mean = round(func_df['CBD'].mean(),0)

    return mean

def complete_cannabinoids_nan(df,col):
    

    # for col in col_names:
    try:
        nanim = df[(df[f'{col}'].isna()) & ((df['Left Parent'].notna()) | (df['Right parent'].notna()))].copy() # To apply changes on real DF

        nanim[f'{col}'] = nanim.apply(lambda x: my_func(df,x['Left Parent']),axis=1)
        nanim[f'{col}'] = nanim.apply(lambda x: my_func(df,x['Right parent']) if pd.isnull(x[f'{col}']) else x[f'{col}'],axis=1)
        # fixed_df.append(nanim)
    except Exception as e:
        # print(f'Didnt get data for {col_name}')
        pass


    # fixed_df = pd.concat(fixed_df)

    return nanim






In [65]:
# curr_df = df[df['CBD'].isna()]
# curr_df.shape

df['CBD'].isna().sum()

5007

In [66]:

nanim_df = complete_cannabinoids_nan(df,'CBD')


In [67]:
df = df.copy()
df.update(nanim_df)

In [68]:
df['CBD'].isna().sum()

3617

In [31]:
curr_df[curr_df['CBD'].notna()]

,Unnamed: 0,Strain Name,Type,Rating,Rating Users,Left Parent,Right parent,Left Child,Right Child,Feeling_1,...,Helps with_3,Feeling_4,Negative_4,Helps with_4,Feeling_5,Negative_5,Helps with_5,THC,CBD,CBG


In [ ]:
df.columns.str.match("Unnamed") ## A new column was created from some reason
df = df.loc[:,~df.columns.str.match("Unnamed")]



In [5]:


# curr_df = df[(df['Feeling_1'].isna()) & (df['Feeling_2'].isna()) & (df['Feeling_3'].isna())]
# curr_df = df[(df['Feeling_1'].isnull()) & (df['Feeling_2'].isnull()) & (df['Feeling_3'].isnull()) & ((df['Left Parent'].notna()) | (df['Right parent'].notna()))]
# curr_df = df[(df['Negative_1'].isnull()) & (df['Negative_2'].isnull()) & (df['Negative_3'].isnull())]
curr_df = df[df['THC'].isna()]
curr_df.shape
# curr_df.head(10)


(1746, 27)

In [123]:
def my_func2(df,parent,mode,col):

    func_df = df.loc[(df['Left Parent'] == parent) | (df['Right parent'] == parent) | (df['Strain Name'] == parent)] ## All rows with same parent, including the parent

    if mode == 'Feelings':
        try:
            feeling = func_df[col].value_counts().idxmax() ## Limits to look only at the same col

        except Exception as e:
            feeling = np.NaN

        return feeling

    elif mode == 'Cannabinoids':

        mean = round(func_df[col].mean(),0)
        
        return mean



def complete_feelings(df,col_names):
        
    # main_df = []
    for i,col in enumerate(col_names,0):
        if 'Feeling' in col:
            nanim = df[(df['Feeling_1'].isnull()) & (df['Feeling_2'].isnull()) & (df['Feeling_3'].isnull()) & ((df['Left Parent'].notna()) | (df['Right parent'].notna()))].copy()
            mode = 'Feelings'

        elif 'Negative' in col:
            nanim = df[(df['Negative_1'].isnull()) & (df['Negative_2'].isnull()) & (df['Negative_3'].isnull()) & ((df['Left Parent'].notna()) | (df['Right parent'].notna()))].copy()
            mode = 'Feelings'
        
        elif 'Helps with' in col:
            nanim = df[(df['Helps with_1'].isnull()) & (df['Helps with_2'].isnull()) & (df['Helps with_3'].isnull()) & ((df['Left Parent'].notna()) | (df['Right parent'].notna()))].copy()
            mode = 'Feelings'
        
        else:
            nanim = df[(df[col].isna()) & ((df['Left Parent'].notna()) | (df['Right parent'].notna()))].copy() # To apply changes on real DF
            mode = 'Cannabinoids'
        
        try:
            nanim[col] = nanim.apply(lambda x: my_func2(df,x['Left Parent'],mode,col),axis=1)
            nanim[col] = nanim.apply(lambda x: my_func2(df,x['Right parent'],mode,col) if pd.isnull(x[col]) else x[col],axis=1)
            
            df[col].fillna(nanim[col],inplace=True)

            # if i == 0: ## First Dataframe
            #     main_df = nanim
            # else: ## Fill NaNs on main Dataframe
            #     main_df[col].fillna(nanim[col],inplace=True)

        except Exception as e:
            print(e)
            pass
    


    # return nanim
    # return main_df
    return df

In [124]:
# cols = ['Feeling_1','Fas']

# if any('Feeling' in s for s in cols):
#     print('Si')

# cols = ['Feelings_1','Feelings_2','Feelings_3']

print(df['Feeling_1'].isna().sum())
print(df['Feeling_2'].isna().sum())
print(df['Feeling_3'].isna().sum())
print(df['Negative_1'].isna().sum())
print(df['Negative_2'].isna().sum())
print(df['Negative_3'].isna().sum())
print(df['Helps with_1'].isna().sum())
print(df['Helps with_2'].isna().sum())
print(df['Helps with_3'].isna().sum())
print(df['THC'].isna().sum())
print(df['CBD'].isna().sum())
print(df['CBG'].isna().sum())


1675
1725
1772
3221
3221
3221
2217
2217
2217
1746
5007
3053


In [125]:
cols = ['Feeling_1','Feeling_2','Feeling_3','Negative_1','Negative_2','Negative_3','Helps with_1','Helps with_2','Helps with_3']
# cols = ['THC']
feelings_complete = complete_feelings(df,cols)

In [126]:
df = df.copy()
df.update(feelings_complete)

In [128]:
print(df['Feeling_1'].isna().sum())
print(df['Feeling_2'].isna().sum())
print(df['Feeling_3'].isna().sum())
print(df['Negative_1'].isna().sum())
print(df['Negative_2'].isna().sum())
print(df['Negative_3'].isna().sum())
print(df['Helps with_1'].isna().sum())
print(df['Helps with_2'].isna().sum())
print(df['Helps with_3'].isna().sum())
print(df['THC'].isna().sum())
print(df['CBD'].isna().sum())
print(df['CBG'].isna().sum())

1404
1725
1772
1790
2781
3221
1626
2118
2217
1746
5007
3053


In [ ]:
# curr_df = df[(df['Feeling_1'].isnull()) & (df['Feeling_2'].isnull()) & (df['Feeling_3'].isnull()) & ((df['Left Parent'].notna()) | (df['Right parent'].notna()))]
feelings_complete.head(15)
# feelings_complete.shape

In [ ]:
df['THC'].hist(bins=30,rwidth=0.8,grid=False)
plt.xlabel('THC%')
plt.ylabel('Amount')
plt.title('THC% Distribution')
plt.show()

In [ ]:

most_freq1 = df['Feeling_1'].value_counts()[:5].sort_index(ascending=True)
most_freq2 = df['Feeling_2'].value_counts()[:5].sort_index(ascending=True)
most_freq3 = df['Feeling_3'].value_counts()[:5].sort_index(ascending=True)


freq = most_freq1 + most_freq2 + most_freq3


freq.plot(kind='pie')
plt.title('Top 5 Feelings')
plt.show()

In [ ]:
## Top health conditions

most_freq1 = df['Helps with_1'].value_counts()[:5].sort_index(ascending=True)
most_freq2 = df['Helps with_2'].value_counts()[:5].sort_index(ascending=True)
most_freq3 = df['Helps with_3'].value_counts()[:5].sort_index(ascending=True)


freq = most_freq1 + most_freq2 + most_freq3


freq.plot(kind='bar')
plt.show()